# Reviews Analysis
## Opening of the csv files

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("inputs_from_processing/comments_cleaned.csv", sep='|')
df_2 = pd.read_csv("inputs_from_processing/basic_data_cleaned.csv", sep='|', index_col=[0])

## Transformation
### Create the last 50 comments rate and add the to df

In [3]:
df_merge = df.groupby(['name', 'address'])['comment_rate'].mean().to_frame()

df_last_average_rate = df_2.merge(df_merge['comment_rate'], how='left', on=['name', 'address'])

### Create boolean column if the average 'last 50' is equal or better than the average rate

In [4]:
df_last_average_rate['last_50_better?'] = np.where(df_last_average_rate['comment_rate'] == df_last_average_rate['average_rate'], "Equal", np.where(df_last_average_rate['comment_rate'] >= df_last_average_rate['average_rate'], "Better", "Worse"))
# df_last_average_rate[df_last_average_rate['location'].isna()]

## Writing CSV file

In [5]:
df_last_average_rate.to_csv("outputs/comments_rates_cleaned.csv", sep='|')

## Analysis
### How many pizzerias are now better than their average rate ? 

In [6]:
df_last_average_rate.groupby(['last_50_better?'])['last_50_better?'].count()

last_50_better?
Better    294
Equal      34
Worse     237
Name: last_50_better?, dtype: int64

### Add the quantile column

In [7]:
df_last_average_rate['decile_rank_average_rate'] = pd.qcut(df_last_average_rate['average_rate'],10, labels = False) + 1
df_last_average_rate['decile_rank_nb_of_reviews'] = pd.qcut(df_last_average_rate['nb_of_reviews'],4, labels = False) + 1
df_last_average_rate['decile_rank_average_commennt_rate'] = pd.qcut(df_last_average_rate['comment_rate'],10, labels = False) + 1
df_last_average_rate

,name,average_rate,nb_of_reviews,address,city,postal_code,comment_rate,last_50_better?,decile_rank_average_rate,decile_rank_nb_of_reviews,decile_rank_average_commennt_rate
0,Pizza Wawa,4.5,1535,"35 Rue Saint-Honoré, 75001 Paris",Paris,75001,4.580000,Better,7,4,7.0
1,Pizzeria Iovine's,4.5,2591,"7bis Rue du Colonel Driant, 75001 Paris",Paris,75001,4.652174,Better,7,4,8.0
2,Milo,4.6,579,"44 Rue Saint-Honoré, 75001 Paris",Paris,75001,4.600000,Equal,8,4,8.0
3,La Tavola Calda,4.5,513,"39 Rue des Bourdonnais, 75001 Paris",Paris,75001,4.440000,Worse,7,3,6.0
4,Liberto,4.7,1697,"23 Rue Berger, 75001 Paris",Paris,75001,4.358974,Worse,9,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...
560,15-12,1.5,6,"6 Av. Rapp, 75007 Paris",Paris,75007,1.500000,Equal,1,1,1.0
561,Nicole's,4.0,426,"14 Rue de Bellechasse, 75007 Paris",Paris,75007,3.360000,Worse,2,3,1.0
562,Pizza Fiorentina,3.0,571,"7 Bd de Grenelle, 75015 Paris",Paris,75015,3.060000,Better,1,4,1.0
563,Primo Piano,3.9,45,"Le Bon Marché, 24 Rue de Sèvres, 75007 Paris",Paris,75007,3.933333,Better,2,1,3.0


### Try of double sorting

In [15]:
df_last_average_rate.sort_values(by=['average_rate', 'nb_of_reviews'], ascending=[False, False]).head(10)

,name,average_rate,nb_of_reviews,address,city,postal_code,comment_rate,last_50_better?,decile_rank_average_rate,decile_rank_nb_of_reviews,decile_rank_average_commennt_rate
116,Univers Pizza,5.0,80,"305 Rue Saint-Jacques, 75005 Paris",Paris,75005,5.00,Equal,10,1,10.0
486,Pizzeria Palermo,5.0,3,"35 Rue Pelleport, 75020 Paris",Paris,75020,5.00,Equal,10,1,10.0
91,PALLA PIZZA,5.0,2,"10 R. des Petites Écuries, 75010 Paris",Paris,75010,5.00,Equal,10,1,10.0
548,Dafu,5.0,2,"22bis Rue Jean Nicot, 75007 Paris",Paris,75007,5.00,Equal,10,1,10.0
48,Pizza Democrazia,5.0,1,"13 Rue Montorgueil, 75001 Paris",Paris,75001,5.00,Equal,10,1,10.0
164,Panuozzo,5.0,1,"11 Rue Cujas, 75005 Paris",Paris,75005,5.00,Equal,10,1,10.0
492,Clip Pizza,5.0,1,"178 Rue de Belleville, 75020 Paris",Paris,75020,5.00,Equal,10,1,10.0
434,Little Capri,4.9,328,"3 Sq. Paul Paray, 75017 Paris",Paris,75017,4.96,Better,10,3,10.0
362,Chez Achille,4.9,175,"137 Av. de Versailles, 75016 Paris",Paris,75016,4.92,Better,10,2,10.0
102,Pizza Omilgout,4.9,109,"3 Rue de Budapest, 75009 Paris",Paris,75009,4.98,Better,10,1,10.0
